In [54]:
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, LearningRateScheduler
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import LeakyReLU

import h5py
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

# GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [30]:
# import the data library
with h5py.File("ProcessedData_loc_96x96x65_b1r_96x96x16_crossValidation.mat", 'r') as f:
   X = f['lvSaveDataInput'][:,:,:,:]
   y = f['lvLovalizerSave'][:,:,:,:]

In [33]:
print(X.shape)
print(y.shape)

(126, 96, 96, 16)
(126, 96, 96, 65)


In [32]:
# move axis
X = np.moveaxis(X, 1, -1)
y = np.moveaxis(y, 1, -1)

In [60]:
# split in training and test data, for model selection we did a 5-fold cross falidation
y_train, y_val, x_train, x_val = train_test_split(X, y, test_size=0.2)

In [61]:
print(x_train.shape)
print(y_train.shape)

(100, 96, 96, 65)
(100, 96, 96, 16)


In [52]:
# define data shape
input_shape=(96,96,65)
output_shape=(96,96,16)

In [53]:
# define loss function
def symm_loss_single(y_true, y_pred):
    P_Z = tf.norm((y_pred[:,:,:,0]*y_true[:,:,:,1]-y_pred[:,:,:,1]*y_true[:,:,:,0]),1)
    P_N =  tf.norm((y_true[:,:,:,0]*y_true[:,:,:,0]+y_true[:,:,:,1]*y_true[:,:,:,1]),1)

    P = P_Z/P_N

    L1 = tf.norm((y_pred-y_true),ord=1)

    return (P+L1)


In [55]:
# define convolutianol block
init = RandomNormal(stddev=0.02)

def C2D_BLock(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer=init,
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer=init,
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x

In [56]:
# define bifurcated UNet
def define_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # downsampling
    down1 = C2D_BLock(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    pool1 = MaxPooling2D((2, 2)) (down1)
    pool1 = Dropout(dropout)(pool1)

    down2 = C2D_BLock(pool1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    pool2 = MaxPooling2D((2, 2)) (down2)
    pool2 = Dropout(dropout)(pool2)

    down3 = C2D_BLock(pool2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    pool3 = MaxPooling2D((2, 2)) (down3)
    pool3 = Dropout(dropout*2)(pool3)

    down4 = C2D_BLock(pool3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    pool4 = MaxPooling2D(pool_size=(2, 2)) (down4)
    pool4 = Dropout(dropout*2)(pool4)
    
    down5 = C2D_BLock(pool4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    down5 = Dropout(dropout*3)(down5)
    
    # upsamping
    up1_1 = Conv2DTranspose(n_filters*8, (2, 2), strides=(2, 2), padding='same') (down5)
    up1_1 = concatenate([up1_1, down4])
    up1_1 = Dropout(dropout*2)(up1_1)
    up1_1 = C2D_BLock(up1_1, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    up2_1 = Conv2DTranspose(n_filters*4, (2, 2), strides=(2, 2), padding='same') (up1_1)
    up2_1 = concatenate([up2_1, down3])
    up2_1 = Dropout(dropout*2)(up2_1)
    up2_1 = C2D_BLock(up2_1, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    up3_1 = Conv2DTranspose(n_filters*2, (2, 2), strides=(2, 2), padding='same') (up2_1)
    up3_1 = concatenate([up3_1, down2])
    up3_1 = Dropout(dropout)(up3_1)
    up3_1 = C2D_BLock(up3_1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    up4_1 = Conv2DTranspose(n_filters*1, (2, 2), strides=(2, 2), padding='same') (up3_1)
    up4_1 = concatenate([up4_1, down1], axis=3)
    up4_1 = Dropout(dropout)(up4_1)
    up4_1 = C2D_BLock(up4_1, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
        
    outputsCh1 = Conv2D(2, (1, 1), activation='tanh', name="outputsCh1") (up4_1)
    
    up1_2 = Conv2DTranspose(n_filters*8, (2, 2), strides=(2, 2), padding='same') (down5)
    up1_2 = concatenate([up1_2, down4])
    up1_2 = Dropout(dropout*2)(up1_2)
    up1_2 = C2D_BLock(up1_2, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    up2_2 = Conv2DTranspose(n_filters*4, (2, 2), strides=(2, 2), padding='same') (up1_2)
    up2_2 = concatenate([up2_2, down3])
    up2_2 = Dropout(dropout*2)(up2_2)
    up2_2 = C2D_BLock(up2_2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    up3_2 = Conv2DTranspose(n_filters*2, (2, 2), strides=(2, 2), padding='same') (up2_2)
    up3_2 = concatenate([up3_2, down2])
    up3_2 = Dropout(dropout)(up3_2)
    up3_2 = C2D_BLock(up3_2, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    up4_2 = Conv2DTranspose(n_filters*1, (2, 2), strides=(2, 2), padding='same') (up3_2)
    up4_2 = concatenate([up4_2, down1], axis=3)
    up4_2 = Dropout(dropout)(up4_2)
    up4_2 = C2D_BLock(up4_2, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputsCh2 = Conv2D(2, (1, 1), activation='tanh', name="outputsCh2") (up4_2)   
    
    up1_3 = Conv2DTranspose(n_filters*8, (2, 2), strides=(2, 2), padding='same') (down5)
    up1_3 = concatenate([up1_3, down4])
    up1_3 = Dropout(dropout*2)(up1_3)
    up1_3 = C2D_BLock(up1_3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    up2_3 = Conv2DTranspose(n_filters*4, (2, 2), strides=(2, 2), padding='same') (up1_3)
    up2_3 = concatenate([up2_3, down3])
    up2_3 = Dropout(dropout*2)(up2_3)
    up2_3 = C2D_BLock(up2_3, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    up3_3 = Conv2DTranspose(n_filters*2, (2, 2), strides=(2, 2), padding='same') (up2_3)
    up3_3 = concatenate([up3_3, down2])
    up3_3 = Dropout(dropout)(up3_3)
    up3_3 = C2D_BLock(up3_3, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    up4_3 = Conv2DTranspose(n_filters*1, (2, 2), strides=(2, 2), padding='same') (up3_3)
    up4_3 = concatenate([up4_3, down1], axis=3)
    up4_3 = Dropout(dropout)(up4_3)
    up4_3 = C2D_BLock(up4_3, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm) 
    
    outputsCh3 = Conv2D(2, (1, 1), activation='tanh', name="outputsCh3") (up4_3)
    
    up1_4 = Conv2DTranspose(n_filters*8, (2, 2), strides=(2, 2), padding='same') (down5)
    up1_4 = concatenate([up1_4, down4])
    up1_4 = Dropout(dropout*2)(up1_4)
    up1_4 = C2D_BLock(up1_4, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    up2_4 = Conv2DTranspose(n_filters*4, (2, 2), strides=(2, 2), padding='same') (up1_4)
    up2_4 = concatenate([up2_4, down3])
    up2_4 = Dropout(dropout*2)(up2_4)
    up2_4 = C2D_BLock(up2_4, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    up3_4 = Conv2DTranspose(n_filters*2, (2, 2), strides=(2, 2), padding='same') (up2_4)
    up3_4 = concatenate([up3_4, down2])
    up3_4 = Dropout(dropout)(up3_4)
    up3_4 = C2D_BLock(up3_4, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    up4_4 = Conv2DTranspose(n_filters*1, (2, 2), strides=(2, 2), padding='same') (up3_4)
    up4_4 = concatenate([up4_4, down1], axis=3)
    up4_4 = Dropout(dropout)(up4_4)
    up4_4 = C2D_BLock(up4_4, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm) 
    
    outputsCh4 = Conv2D(2, (1, 1), activation='tanh', name="outputsCh4") (up4_4)
    
    up1_5 = Conv2DTranspose(n_filters*8, (2, 2), strides=(2, 2), padding='same') (down5)
    up1_5 = concatenate([up1_5, down4])
    up1_5 = Dropout(dropout*2)(up1_5)
    up1_5 = C2D_BLock(up1_5, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    up2_5 = Conv2DTranspose(n_filters*4, (2, 2), strides=(2, 2), padding='same') (up1_5)
    up2_5 = concatenate([up2_5, down3])
    up2_5 = Dropout(dropout*2)(up2_5)
    up2_5 = C2D_BLock(up2_5, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    up3_5 = Conv2DTranspose(n_filters*2, (2, 2), strides=(2, 2), padding='same') (up2_5)
    up3_5 = concatenate([up3_5, down2])
    up3_5 = Dropout(dropout)(up3_5)
    up3_5 = C2D_BLock(up3_5, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    up4_5 = Conv2DTranspose(n_filters*1, (2, 2), strides=(2, 2), padding='same') (up3_5)
    up4_5 = concatenate([up4_5, down1], axis=3)
    up4_5 = Dropout(dropout)(up4_5)
    up4_5 = C2D_BLock(up4_5, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputsCh5 = Conv2D(2, (1, 1), activation='tanh', name="outputsCh5") (up4_5)
    
    up1_6 = Conv2DTranspose(n_filters*8, (2, 2), strides=(2, 2), padding='same') (down5)
    up1_6 = concatenate([up1_6, down4])
    up1_6 = Dropout(dropout*2)(up1_6)
    up1_6 = C2D_BLock(up1_6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    up2_6 = Conv2DTranspose(n_filters*4, (2, 2), strides=(2, 2), padding='same') (up1_6)
    up2_6 = concatenate([up2_6, down3])
    up2_6 = Dropout(dropout*2)(up2_6)
    up2_6 = C2D_BLock(up2_6, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    up3_6 = Conv2DTranspose(n_filters*2, (2, 2), strides=(2, 2), padding='same') (up2_6)
    up3_6 = concatenate([up3_6, down2])
    up3_6 = Dropout(dropout)(up3_6)
    up3_6 = C2D_BLock(up3_6, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    up4_6 = Conv2DTranspose(n_filters*1, (2, 2), strides=(2, 2), padding='same') (up3_6)
    up4_6 = concatenate([up4_6, down1], axis=3)
    up4_6 = Dropout(dropout)(up4_6)
    up4_6 = C2D_BLock(up4_6, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputsCh6 = Conv2D(2, (1, 1), activation='tanh', name="outputsCh6") (up4_6)
    
    up1_7 = Conv2DTranspose(n_filters*8, (2, 2), strides=(2, 2), padding='same') (down5)
    up1_7 = concatenate([up1_7, down4])
    up1_7 = Dropout(dropout*2)(up1_7)
    up1_7 = C2D_BLock(up1_7, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    up2_7 = Conv2DTranspose(n_filters*4, (2, 2), strides=(2, 2), padding='same') (up1_7)
    up2_7 = concatenate([up2_7, down3])
    up2_7 = Dropout(dropout*2)(up2_7)
    up2_7 = C2D_BLock(up2_7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    up3_7 = Conv2DTranspose(n_filters*2, (2, 2), strides=(2, 2), padding='same') (up2_7)
    up3_7 = concatenate([up3_7, down2])
    up3_7 = Dropout(dropout)(up3_7)
    up3_7 = C2D_BLock(up3_7, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    up4_7 = Conv2DTranspose(n_filters*1, (2, 2), strides=(2, 2), padding='same') (up3_7)
    up4_7 = concatenate([up4_7, down1], axis=3)
    up4_7 = Dropout(dropout)(up4_7)
    up4_7 = C2D_BLock(up4_7, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputsCh7 = Conv2D(2, (1, 1), activation='tanh', name="outputsCh7") (up4_7)
    
    up1_8 = Conv2DTranspose(n_filters*8, (2, 2), strides=(2, 2), padding='same') (down5)
    up1_8 = concatenate([up1_8, down4])
    up1_8 = Dropout(dropout*2)(up1_8)
    up1_8 = C2D_BLock(up1_8, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    up2_8 = Conv2DTranspose(n_filters*4, (2, 2), strides=(2, 2), padding='same') (up1_8)
    up2_8 = concatenate([up2_8, down3])
    up2_8 = Dropout(dropout*2)(up2_8)
    up2_8 = C2D_BLock(up2_8, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    up3_8 = Conv2DTranspose(n_filters*2, (2, 2), strides=(2, 2), padding='same') (up2_8)
    up3_8 = concatenate([up3_8, down2])
    up3_8 = Dropout(dropout)(up3_8)
    up3_8 = C2D_BLock(up3_8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    up4_8 = Conv2DTranspose(n_filters*1, (2, 2), strides=(2, 2), padding='same') (up3_8)
    up4_8 = concatenate([up4_8, down1], axis=3)
    up4_8 = Dropout(dropout)(up4_8)
    up4_8 = C2D_BLock(up4_8, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputsCh8 = Conv2D(2, (1, 1), activation='tanh', name="outputsCh8") (up4_8)



    
    model = Model(inputs=[input_img], outputs=[outputsCh1, outputsCh2, outputsCh3, outputsCh4, outputsCh5, outputsCh6, outputsCh7, outputsCh8])
    return model

In [62]:
# biuld neural network
# learning rate decay and saving of checkpoints is defined
input_img = Input(input_shape)
model = define_unet(input_img, n_filters=32, dropout=0.3, batchnorm=True)
optimizer = Adam(learning_rate=1e-04, beta_1=0.9, beta_2=0.999, epsilon=1e-07,clipnorm=1.0, amsgrad=False)


def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
       return optimizer.lr
    return lr

def lr_scheduler(epoch, lr):
    decay_rate = 0.99944 
    decay_step = 1
    if epoch % decay_step == 0 and epoch:
        return lr * decay_rate
    return lr

class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(
            "loss: {:.4e} - "
            "val_loss: {:.4e}".format(
                logs["loss"], logs["val_loss"]
            )
        )

## checkpoint
filepath = "checks.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min', period=100)
callbacks_list = [checkpoint, LearningRateScheduler(lr_scheduler),CustomCallback()]
lr_metric = get_lr_metric(optimizer)

model.compile(optimizer=optimizer, loss=symm_loss_single,  metrics=['mse', lr_metric])
model.summary()


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 96, 96, 65)] 0                                            
__________________________________________________________________________________________________
conv2d_528 (Conv2D)             (None, 96, 96, 32)   18752       input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_528 (BatchN (None, 96, 96, 32)   128         conv2d_528[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_528 (LeakyReLU)     (None, 96, 96, 32)   0           batch_normalization_528[0][0]    
____________________________________________________________________________________________

In [63]:
# training of the model
history = model.fit(x=x_train, 
                    y={"outputsCh1": y_train[:,:,:,0:2],"outputsCh2": y_train[:,:,:,2:4],"outputsCh3": y_train[:,:,:,4:6],"outputsCh4": y_train[:,:,:,6:8],"outputsCh5": y_train[:,:,:,8:10],"outputsCh6": y_train[:,:,:,10:12],"outputsCh7": y_train[:,:,:,12:14],"outputsCh8": y_train[:,:,:,14:16]},                                      
                    validation_data=(x_val, {"outputsCh1": y_val[:,:,:,0:2],"outputsCh2": y_val[:,:,:,2:4],"outputsCh3": y_val[:,:,:,4:6],"outputsCh4": y_val[:,:,:,6:8],"outputsCh5": y_val[:,:,:,8:10],"outputsCh6": y_val[:,:,:,10:12],"outputsCh7": y_val[:,:,:,12:14],"outputsCh8": y_val[:,:,:,14:16]}),
                    shuffle=True, epochs=4000, 
                    batch_size=2, callbacks=callbacks_list)                
                    #callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

Epoch 1/4000
50/50 [==============================] - 7s 138ms/step - loss: 44720.7461 - outputsCh1_loss: 5507.0205 - outputsCh2_loss: 4796.6143 - outputsCh3_loss: 4965.4233 - outputsCh4_loss: 7297.5850 - outputsCh5_loss: 5578.7598 - outputsCh6_loss: 5982.9785 - outputsCh7_loss: 4734.5430 - outputsCh8_loss: 5857.8198 - outputsCh1_mse: 0.0897 - outputsCh1_lr: 1.0000e-04 - outputsCh2_mse: 0.0644 - outputsCh2_lr: 1.0000e-04 - outputsCh3_mse: 0.0702 - outputsCh3_lr: 1.0000e-04 - outputsCh4_mse: 0.1301 - outputsCh4_lr: 1.0000e-04 - outputsCh5_mse: 0.0832 - outputsCh5_lr: 1.0000e-04 - outputsCh6_mse: 0.1018 - outputsCh6_lr: 1.0000e-04 - outputsCh7_mse: 0.0638 - outputsCh7_lr: 1.0000e-04 - outputsCh8_mse: 0.0964 - outputsCh8_lr: 1.0000e-04 - val_loss: 7364.1729 - val_outputsCh1_loss: 1093.6132 - val_outputsCh2_loss: 812.9810 - val_outputsCh3_loss: 1089.4640 - val_outputsCh4_loss: 864.1830 - val_outputsCh5_loss: 935.3535 - val_outputsCh6_loss: 700.9221 - val_outputsCh7_loss: 779.6199 - val_out

KeyboardInterrupt: 

In [ ]:
# load weights of the pretrained network on all thorax geoemtries
model.load_weights("Weights_split8SepDeCode_K32_leakyIni_0p3drop_BN_1e4Conv_commPha3Chan_2batch_mse_4000epochs_CV2.h5")

In [64]:
# load data for in vivo application
with h5py.File("ProcessedData_loc_96x96x65_b1r_96x96x16_testData.mat", 'r') as f:
   y_test = f['lvSaveDataInput'][:,:,:,:]
   x_test = f['lvLovalizerSave'][:,:,:]

In [65]:
# move axis
x_test = np.moveaxis(x_test, 1, -1)
y_test = np.moveaxis(y_test, 1, -1)

In [66]:
# do prediction
P_test= np.array(model.predict(x_test))